In [ ]:
import pandas as pd
import numpy as np
import os, sys, leidenalg, pickle
from scipy.stats import spearmanr
from statsmodels.sandbox.stats.multicomp import multipletests
import igraph as ig

In [ ]:
class Network_Analysis:
    def __init__(self,raw_data,nodes,respath,cat):
        self.res_path=respath
        self.cat=cat
        if os.path.isdir(self.res_path):
            pass
        else:
            os.mkdir(self.res_path)
        self.network_ori=self.__calc(raw_data)
        self.nodes=nodes

        print('Network Analysis')
        self.__net_analysis_combi()
    
    def __calc(self,df):
        print('Calculating Correlation..')
        temp=spearmanr(df.T)
        corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
        pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        res=res[['level_0','level_1','weight','pval']]
        res['category'] = self.cat
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res=res[res.padj < 0.05].reset_index(drop=True)
        res.columns=['source','target','correlation','pvalue','category','padj']
        res=res[['source','target','category','correlation','pvalue','padj']]
        res.to_csv('%s/%d_edges.txt' % (self.res_path,self.cat),sep='\t',index=False)
        print('Done!!')
        return res
    
    def __net_analysis_combi(self):
        print('Loading The Network...')
        temp=self.network_ori
        g= ig.Graph.TupleList(zip(temp['source'],temp['target'],temp['correlation']),weights=True)
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        self.clustering_combi=pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        print('Cluster Analysis...')
        self.modularity_combi=diff
        open('%s/%d_modularity.txt' % (self.res_path,self.cat),'w').write(str(diff))
        self.nodes['cluster'] = self.clustering_combi.reindex(self.nodes.index).tolist()
        self.nodes.to_csv('%s/%d_nodes.txt' % (self.res_path,self.cat),sep='\t')
    
    def save_network(self):
        pickle_out = open('%s/%d_network_object.pkl' % (self.res_path,self.cat),"wb")
        pickle.dump(self, pickle_out)
        pickle_out.close()

In [ ]:
print('StartingNet')
k=Network_Analysis(raw_data=normalized_count,nodes=list_of_nodes,respath='../PathToStoreResults',cat=categoryNumber)
print('SavingNet')
k.save_network()